# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
#env = UnityEnvironment(file_name="Tennis_Linux/Tennis.x86_64")
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
#from buffer import ReplayBuffer
from common.Memory import ReplayMemory
from maddpg import MADDPG
import torch
import numpy as np
from tensorboardX import SummaryWriter
import os
from utilities import transpose_list, transpose_to_tensor
from collections import deque

# keep training awake
#from workspace_utils import keep_awake

# for saving gif
#import imageio

%load_ext autoreload
%autoreload 2

In [6]:
seed = 12345
np.random.seed(seed)
torch.manual_seed(seed)

# number of parallel agents
#parallel_envs = 4
# number of training episodes.
# change this to higher number to experiment. say 30000.
number_of_episodes = 2000
episode_length = 200
batchsize = 128
# how many episodes to save policy and gif
save_interval = 1000
t = 0

In [7]:
# amplitude of OU noise
# this slowly decreases to 0
noise = 6
noise_reduction = 0.9999

# how many episodes before update
episode_per_update = 10

log_path = os.getcwd()+"/log"
model_dir= os.getcwd()+"/model_dir"

os.makedirs(model_dir, exist_ok=True)

#torch.set_num_threads(parallel_envs)
#env = envs.make_parallel_env(parallel_envs)

# keep 5000 episodes worth of replay
buffer = ReplayMemory(int(1e5))

# initialize policy and critic

in_actor = state_size
hidden_in_actor = 256
hidden_out_actor = 256
out_actor = 2
# critic input = obs from both agents + actions from both agents
in_critic = 2*state_size + 2*action_size
hidden_in_critic = 512
hidden_out_critic = 256

maddpg = MADDPG(in_actor, hidden_in_actor, hidden_out_actor, 
                out_actor, in_critic, hidden_in_critic, hidden_out_critic)
logger = SummaryWriter(log_dir=log_path)
agent0_reward = []
agent1_reward = []

In [8]:
# training loop
# show progressbar
import progressbar as pb
widget = ['episode: ', pb.Counter(),'/',str(number_of_episodes),' ', 
          pb.Percentage(), ' ', pb.ETA(), ' ', pb.Bar(marker=pb.RotatingMarker()), ' ' ]

timer = pb.ProgressBar(widgets=widget, maxval=number_of_episodes).start()
'''
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
actions = np.random.randn(num_agents, action_size)
actions = np.clip(actions, -1, 1)
env_info = env.step(actions)[brain_name] 
'''

scores_deque = deque(np.zeros(100))

# initialize buffer with random actions

for _ in range(1500):
    env_info = env.reset(train_mode=True)[brain_name]
    obs = env_info.vector_observations
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    actions = actions.astype(np.float32)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_obs = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    
    # add data to buffer
    transition = (obs, actions, rewards, next_obs, dones)
    buffer.push(*transition)

for episode in range(0, number_of_episodes):

    timer.update(episode)

    reward_this_episode = np.zeros(num_agents)
    env_info = env.reset(train_mode=True)[brain_name]
    obs = env_info.vector_observations

    #for calculating rewards for this particular episode - addition of all time steps

    # save info or not
    save_info = ((episode) % save_interval == 0 or episode==number_of_episodes-1)
    #frames = []
    tmax = 0
    '''
    if save_info:
        frames.append(env.render('rgb_array'))
    '''
    #for episode_t in range(episode_length):
    while True:
        #t += 1 #parallel_envs

        # explore = only explore for a certain number of episodes
        # action input needs to be transposed
        actions = maddpg.act(torch.tensor(obs, dtype=torch.float), noise=noise)
        noise *= noise_reduction

        actions = torch.stack(actions).detach().numpy()

        # step forward one frame
        env_info = env.step(actions)[brain_name]
        next_obs = env_info.vector_observations            # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
 
        # add data to buffer
        transition = (obs, actions, rewards, next_obs, dones)
        buffer.push(*transition)

        reward_this_episode += rewards

        obs = next_obs
        '''
        # save gif frame
        if save_info:
            frames.append(env.render('rgb_array'))
            tmax+=1
        '''
        if np.any(dones):
            break
    
    scores_deque.append(np.sum(reward_this_episode))
    
    # update once after every episode_per_update
    if len(buffer) > batchsize and episode % episode_per_update == 0:
        samples = buffer.sample(batchsize)
        for a_i in range(num_agents):
            #samples = buffer.sample(batchsize)
            maddpg.update(samples, a_i, logger)
        maddpg.update_targets() #soft update the target network towards the actual networks

    agent0_reward.append(reward_this_episode[0])
    agent1_reward.append(reward_this_episode[1])

    if episode % 100 == 0 or episode == number_of_episodes-1:
        avg_rewards = [np.mean(agent0_reward), np.mean(agent1_reward)]
        agent0_reward = []
        agent1_reward = []
        for a_i, avg_rew in enumerate(avg_rewards):
            logger.add_scalar('agent%i/mean_episode_rewards' % a_i, avg_rew, episode)
        print('last 100 avg reward {} is {}'.format(episode, np.mean(scores_deque)))

    #saving model
    save_dict_list =[]
    if save_info:
        for i in range(2):

            save_dict = {'actor_params' : maddpg.maddpg_agent[i].actor.state_dict(),
                         'actor_optim_params': maddpg.maddpg_agent[i].actor_optimizer.state_dict(),
                         'critic_params' : maddpg.maddpg_agent[i].critic.state_dict(),
                         'critic_optim_params' : maddpg.maddpg_agent[i].critic_optimizer.state_dict()}
            save_dict_list.append(save_dict)

            torch.save(save_dict_list, 
                       os.path.join(model_dir, 'episode-{}.pt'.format(episode)))
        '''
        # save gif files
        imageio.mimsave(os.path.join(model_dir, 'episode-{}.gif'.format(episode)), 
                        frames, duration=.04)
        '''
        
logger.close()
timer.finish()
env.close()

last 100 avg reward 0 is -9.900989877705526e-05


last 100 avg reward 25 is -0.0004761904064151976


last 100 avg reward 50 is 0.0005960266247687751


last 100 avg reward 75 is 0.00022727289152416316


last 100 avg reward 100 is -0.0005472635025557


...

last 100 avg reward 1900 is -0.005102448496801862


last 100 avg reward 1925 is -0.005162882249019047


last 100 avg reward 1950 is -0.005221842725950587


last 100 avg reward 1975 is -0.005279383152859691


episode: 2000/2000 100% Time: 0:02:04 |||||||||||||||||||||||||||||||||||||||| 
